In [362]:
from PIL import Image
import numpy as np
import os
import cv2
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
import pandas as pd
import sys
%matplotlib inline
from scipy.spatial.distance import euclidean as euc
import matplotlib.pyplot as plt
import random
import plotly.express as px
import numpy
import tensorflow as tf
import requests

In [363]:
def readData(filepath, label):
    cells = []
    labels = []
    file = os.listdir(filepath)
    for img in file:
        try:
            image = cv2.imread(filepath + img)
            image_from_array = Image.fromarray(image, 'RGB')
            size_image = image_from_array.resize((50, 50))
            cells.append(np.array(size_image))
            labels.append(label)
        except AttributeError as e:
            print('Skipping file: ', img, e)
    print(len(cells), ' Data Points Read!')
    return np.array(cells), np.array(labels)

In [364]:
def update(name, Cells, Labels, globalId):
    
    s = np.arange(Cells.shape[0])
    np.random.shuffle(s)
    Cells = Cells[s]
    Labels = Labels[s]
    
    num_classes=len(np.unique(Labels))
    len_data=len(Cells)
    print(len_data, ' Data Points')
    
    (x_train)=Cells
    (y_train)=Labels
    
    # Since we're working on image data, we normalize data by divinding 255.
    x_train = x_train.astype('float32')/255 
    train_len=len(x_train)
    
    #Doing One hot encoding as classifier has multiple classes
    y_train=keras.utils.to_categorical(y_train,num_classes)
    
    #creating sequential model
    model=Sequential()
    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(2,activation="softmax"))#2 represent output layer neurons 
    # model.summary()
    
    if globalId != 1:
        model.load_weights("./weights/global"+str(globalId)+".h5")

    # compile the model with loss as categorical_crossentropy and using adam optimizer
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    #Fit the model with min batch size as 50[can tune batch size to some factor of 2^power ] 
    model.fit(x_train, y_train, batch_size=10, epochs=3, verbose=1)
    
    #Saving Model
    model.save("./weights/"+str(name)+".h5")
    return len_data, model

In [365]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = numpy.random.permutation(len(a))
    return a[p], b[p]

In [366]:
print('Reading Training Data')
ParasitizedCells, ParasitizedLabels = readData('./input/cell_images/Parasitized/', 1)
UninfectedCells, UninfectedLabels  = readData('./input/cell_images/Uninfected/', 0)

Reading Training Data
Skipping file:  Thumbs.db 'NoneType' object has no attribute '__array_interface__'
13779  Data Points Read!
Skipping file:  Thumbs.db 'NoneType' object has no attribute '__array_interface__'
13779  Data Points Read!


In [367]:
Cells = np.concatenate((ParasitizedCells, UninfectedCells))
Labels = np.concatenate((ParasitizedLabels, UninfectedLabels))
Cells, Labels = unison_shuffled_copies(Cells, Labels)

In [368]:
def getDataLen(trainingDict):
    n = 0
    for w in trainingDict:
#         print(w)
        n += trainingDict[w]
    print('Total number of data points after this round: ', n)
    return n

def assignWeights(trainingDf, trainingDict):
    n = getDataLen(trainingDict)
    trainingDf['Weightage'] = trainingDf['DataSize'].apply(lambda x: x/n)
    return trainingDf, n
    
def scale(weight, scaler):
    scaledWeights = []
    for i in range(len(weight)):
        scaledWeights.append(scaler * weight[i])
    return scaledWeights

def getWeight(d):
    #creating sequential model
    model=Sequential()
    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(2,activation="softmax"))#2 represent output layer neurons 
    # model.summary()
    
    fpath = "./weights/"+d+".h5"
    model.load_weights(fpath)
    weight = model.get_weights()
    return weight

def getScaledWeight(d, scaler):
    #creating sequential model
    model=Sequential()
    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(2,activation="softmax"))#2 represent output layer neurons 
    # model.summary()
    
    fpath = "./weights/"+d+".h5"
    model.load_weights(fpath)
    weight = model.get_weights()
    return scale(weight, scaler)

def avgWeights(scaledWeights):
    avg = list()
    for weight_list_tuple in zip(*scaledWeights):
        layer_mean = tf.math.reduce_sum(weight_list_tuple, axis=0)
        avg.append(layer_mean)
    return avg

def FedAvg(trainingDict):
    trainingDf = pd.DataFrame.from_dict(trainingDict, orient='index', columns=['DataSize']) 
    models = list(trainingDict.keys())
    scaledWeights = []
    trainingDf, dataLen = assignWeights(trainingDf, trainingDict)
    for m in models:
        scaledWeights.append(getScaledWeight(m, trainingDf.loc[m]['Weightage']))
    fedAvgWeight = avgWeights(scaledWeights)
    return fedAvgWeight, dataLen

def saveModel(weight, n):
    
    print('Reading Testing Data')
    
    TestParasitizedCells, TestParasitizedLabels = readData('./input/fed/test/Parasitized/', 1)
    TestUninfectedCells, TestUninfectedLabels  = readData('./input/fed/test/Uninfected/', 0)
    TestCells = np.concatenate((TestParasitizedCells, TestUninfectedCells))
    TestLabels = np.concatenate((TestParasitizedLabels, TestUninfectedLabels))
    
    sTest = np.arange(TestCells.shape[0])
    np.random.shuffle(sTest)
    TestCells = TestCells[sTest]
    TestLabels = TestLabels[sTest]
    
    num_classes=len(np.unique(TestLabels))
    
    (x_test) = TestCells
    (y_test) = TestLabels
    
    # Since we're working on image data, we normalize data by divinding 255.
    x_test = x_test.astype('float32')/255
    test_len=len(x_test)
    
    #Doing One hot encoding as classifier has multiple classes
    y_test=keras.utils.to_categorical(y_test,num_classes)

    #creating sequential model
    model=Sequential()
    model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(500,activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(2,activation="softmax"))#2 represent output layer neurons 
    # model.summary()
    
    model.set_weights(weight)

    # compile the model with loss as categorical_crossentropy and using adam optimizer
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    scores = model.evaluate(x_test, y_test)
    print("Loss: ", scores[0])        #Loss
    print("Accuracy: ", scores[1])    #Accuracy

    #Saving Model
    fpath = "./weights/global"+str(n)+".h5"
    model.save(fpath)
    return scores[0], scores[1]

def euclidean(m, n):
    distance = []
    for i in range(len(m)):
#         print(i)
        distance.append(euc(m[i].reshape(-1,1), n[i].reshape(-1,1)))
#     print(distance)
    distance = sum(distance)/len(m)
    return distance

def merge(trainingDict, b):
#     print(trainingDict)
    models = list(trainingDict.keys())
#     print(models)
    trainingDf = pd.DataFrame.from_dict(trainingDict, orient='index', columns=['DataSize'])
    l_weights = []
    g_weight = {}
#     print(models)
    for m in models:
#         print(m)
        if 'global' in m:
            g_weight['name'] = m
            g_weight['weight'] = getWeight(m)
        else:
            l_weights.append({
                'name': m,
                'weight': getWeight(m)
            })
#     print(g_weight)
    scores = {}
            
    for m in l_weights:
        scores[m['name']] = euclidean(m['weight'], g_weight['weight'])

    sortedScores = {k: v for k, v in sorted(scores.items(), key=lambda item: item[1])}
    
#     print(scores)
#     print(sortedScores)

    b = int(len(scores)*b)
    
    selected = []
    
    for i in range(b):
        selected.append((sortedScores.popitem())[0])

    newDict = {}
    for i in trainingDict.keys():
        if (i not in selected) and ('global' not in i):
            newDict[i] = trainingDict[i]

    print('Selections: ', newDict)
    
    NewGlobal, dataLen = FedAvg(newDict)
    
    return NewGlobal, dataLen
    

In [369]:
per_client_batch_size = 100

In [370]:
curr_local = 0
curr_global = 0

In [371]:
local = {}
loss_array = []
acc_array = []
for i in range(0, len(Cells), per_client_batch_size):
    if int(curr_global) == 0:
        curr_global += 1
        name = 'global' + str(curr_global)
        l, m = update(name, Cells[i:i+100], Labels[i:i+100], curr_global)
        local[name] = l
    elif (curr_local != 0) and (int(curr_local)%5 == 0):
        curr_global += 1
        print('Current Global: ', curr_global)
        name = 'global' + str(curr_global)
        m, l = merge(local, 0.25)
        loss, acc = saveModel(m, curr_global)
        loss_array.append(loss)
        acc_array.append(acc)
        curr_local += 1
        local = {}
        local[name] = l
    else:
        print('Current Local: ', curr_local)
        name = str('local'+str(curr_local))
        curr_local += 1
        l, m = update(name, Cells[i:i+100], Labels[i:i+100], curr_global)
        local[name] = l

100  Data Points
Epoch 1/3
10/10 [==============================] - 1s 51ms/step - loss: 0.6985 - accuracy: 0.5400
Epoch 2/3
10/10 [==============================] - 0s 33ms/step - loss: 0.7141 - accuracy: 0.5200
Epoch 3/3
10/10 [==============================] - 0s 34ms/step - loss: 0.6815 - accuracy: 0.5900
Current Local:  0
100  Data Points
Epoch 1/3
10/10 [==============================] - 1s 56ms/step - loss: 0.7425 - accuracy: 0.4600
Epoch 2/3
10/10 [==============================] - 0s 35ms/step - loss: 0.6938 - accuracy: 0.5300
Epoch 3/3
10/10 [==============================] - 0s 35ms/step - loss: 0.6807 - accuracy: 0.6400
Current Local:  1
100  Data Points
Epoch 1/3
10/10 [==============================] - 1s 55ms/step - loss: 0.7691 - accuracy: 0.5700
Epoch 2/3
10/10 [==============================] - 1s 51ms/step - loss: 0.6952 - accuracy: 0.5600
Epoch 3/3
10/10 [==============================] - 0s 33ms/step - loss: 0.6860 - accuracy: 0.5900
Current Local:  2
100  Data Poi

Epoch 2/3
10/10 [==============================] - 0s 38ms/step - loss: 0.6936 - accuracy: 0.4900
Epoch 3/3
10/10 [==============================] - 0s 36ms/step - loss: 0.6937 - accuracy: 0.4900
Current Local:  22
100  Data Points
Epoch 1/3
10/10 [==============================] - 0s 32ms/step - loss: 0.6980 - accuracy: 0.5000
Epoch 2/3
10/10 [==============================] - 0s 45ms/step - loss: 0.6919 - accuracy: 0.5000
Epoch 3/3
10/10 [==============================] - 0s 43ms/step - loss: 0.6918 - accuracy: 0.5000
Current Local:  23
100  Data Points
Epoch 1/3
10/10 [==============================] - 1s 51ms/step - loss: 0.6943 - accuracy: 0.4500
Epoch 2/3
10/10 [==============================] - 0s 30ms/step - loss: 0.6931 - accuracy: 0.5000
Epoch 3/3
10/10 [==============================] - 0s 31ms/step - loss: 0.6908 - accuracy: 0.5500
Current Local:  24
100  Data Points
Epoch 1/3
10/10 [==============================] - 0s 47ms/step - loss: 0.6967 - accuracy: 0.5000
Epoch 2/3


Current Local:  46
100  Data Points
Epoch 1/3
10/10 [==============================] - 0s 30ms/step - loss: 0.6911 - accuracy: 0.5500
Epoch 2/3
10/10 [==============================] - 0s 31ms/step - loss: 0.6831 - accuracy: 0.5500
Epoch 3/3
10/10 [==============================] - 0s 30ms/step - loss: 0.6817 - accuracy: 0.5500
Current Local:  47
100  Data Points
Epoch 1/3
10/10 [==============================] - 0s 40ms/step - loss: 0.6956 - accuracy: 0.5000
Epoch 2/3
10/10 [==============================] - 0s 39ms/step - loss: 0.6844 - accuracy: 0.5600
Epoch 3/3
10/10 [==============================] - 0s 44ms/step - loss: 0.6853 - accuracy: 0.5600
Current Local:  48
100  Data Points
Epoch 1/3
10/10 [==============================] - 0s 35ms/step - loss: 0.6927 - accuracy: 0.5400
Epoch 2/3
10/10 [==============================] - 0s 37ms/step - loss: 0.6921 - accuracy: 0.4700
Epoch 3/3
10/10 [==============================] - 0s 37ms/step - loss: 0.6811 - accuracy: 0.6200
Current Lo

Reading Testing Data
2740  Data Points Read!
2783  Data Points Read!
173/173 [==============================] - 2s 13ms/step - loss: 0.5997 - accuracy: 0.6864
Loss:  0.5997427105903625
Accuracy:  0.6864023208618164
Current Local:  96
100  Data Points
Epoch 1/3
10/10 [==============================] - 0s 28ms/step - loss: 0.6261 - accuracy: 0.6100
Epoch 2/3
10/10 [==============================] - 0s 31ms/step - loss: 0.5999 - accuracy: 0.6500
Epoch 3/3
10/10 [==============================] - 0s 30ms/step - loss: 0.5593 - accuracy: 0.7500
Current Local:  97
100  Data Points
Epoch 1/3
10/10 [==============================] - 0s 21ms/step - loss: 0.6241 - accuracy: 0.6400
Epoch 2/3
10/10 [==============================] - 0s 24ms/step - loss: 0.6623 - accuracy: 0.5900
Epoch 3/3
10/10 [==============================] - 0s 25ms/step - loss: 0.6247 - accuracy: 0.6700
Current Local:  98
100  Data Points
Epoch 1/3
10/10 [==============================] - 0s 28ms/step - loss: 0.6534 - accuracy

10/10 [==============================] - 0s 29ms/step - loss: 0.4174 - accuracy: 0.8200
Current Local:  144
100  Data Points
Epoch 1/3
10/10 [==============================] - 0s 21ms/step - loss: 0.5275 - accuracy: 0.7400
Epoch 2/3
10/10 [==============================] - 0s 23ms/step - loss: 0.5233 - accuracy: 0.7800
Epoch 3/3
10/10 [==============================] - 0s 23ms/step - loss: 0.4604 - accuracy: 0.7900
Current Global:  30
Selections:  {'local141': 100, 'local142': 100, 'local143': 100}
Total number of data points after this round:  300
Reading Testing Data
2740  Data Points Read!
2783  Data Points Read!
173/173 [==============================] - 2s 12ms/step - loss: 0.5070 - accuracy: 0.7498
Loss:  0.5070445537567139
Accuracy:  0.7497736811637878
Current Local:  146
100  Data Points
Epoch 1/3
10/10 [==============================] - 0s 21ms/step - loss: 0.5223 - accuracy: 0.7300
Epoch 2/3
10/10 [==============================] - 0s 25ms/step - loss: 0.4872 - accuracy: 0.77

10/10 [==============================] - 0s 23ms/step - loss: 0.3330 - accuracy: 0.8900
Epoch 3/3
10/10 [==============================] - 0s 21ms/step - loss: 0.2974 - accuracy: 0.9200
Current Local:  193
100  Data Points
Epoch 1/3
10/10 [==============================] - 0s 23ms/step - loss: 0.3663 - accuracy: 0.8800
Epoch 2/3
10/10 [==============================] - 0s 26ms/step - loss: 0.3789 - accuracy: 0.8600
Epoch 3/3
10/10 [==============================] - 0s 29ms/step - loss: 0.3167 - accuracy: 0.9100
Current Local:  194
100  Data Points
Epoch 1/3
10/10 [==============================] - 0s 25ms/step - loss: 0.3906 - accuracy: 0.8700
Epoch 2/3
10/10 [==============================] - 0s 27ms/step - loss: 0.2977 - accuracy: 0.9000
Epoch 3/3
10/10 [==============================] - 0s 25ms/step - loss: 0.3735 - accuracy: 0.9000
Current Global:  40
Selections:  {'local192': 100, 'local193': 100, 'local194': 100}
Total number of data points after this round:  300
Reading Testing 

10/10 [==============================] - 0s 24ms/step - loss: 0.2217 - accuracy: 0.9200
Epoch 2/3
10/10 [==============================] - 0s 24ms/step - loss: 0.1415 - accuracy: 0.9300
Epoch 3/3
10/10 [==============================] - 0s 21ms/step - loss: 0.1221 - accuracy: 0.9700
Current Local:  242
100  Data Points
Epoch 1/3
10/10 [==============================] - 0s 24ms/step - loss: 0.1873 - accuracy: 0.9800
Epoch 2/3
10/10 [==============================] - 0s 28ms/step - loss: 0.0955 - accuracy: 0.9700
Epoch 3/3
10/10 [==============================] - 0s 28ms/step - loss: 0.0808 - accuracy: 0.9800
Current Local:  243
100  Data Points
Epoch 1/3
10/10 [==============================] - 0s 19ms/step - loss: 0.3438 - accuracy: 0.8800
Epoch 2/3
10/10 [==============================] - 0s 22ms/step - loss: 0.2416 - accuracy: 0.8800
Epoch 3/3
10/10 [==============================] - 0s 22ms/step - loss: 0.2033 - accuracy: 0.9200
Current Local:  244
100  Data Points
Epoch 1/3
10/10 [

In [ ]:
#accuracy
fig = px.line(y=acc_array)
fig.show()

In [378]:
#loss
fig = px.line(y=loss_array)
fig.show()